In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
from dotenv import find_dotenv, load_dotenv

_ = load_dotenv(find_dotenv())

In [2]:
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader(input_files = ["eBook-How-to-Build-a-Career-in-AI.pdf"]).load_data()

print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]))
print(documents[0])

<class 'list'> 

41 

<class 'llama_index.schema.Document'>
Doc ID: 75186f0e-07c7-48f7-b2d0-8dd1158203d5
Text: PAGE 1Founder, DeepLearning.AICollected Insights from Andrew Ng
How to  Build Your Career in AIA Simple Guide


In [3]:
from llama_index import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

In [4]:
from llama_index.node_parser import SentenceWindowNodeParser

node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_text"
)

In [5]:
text = "hello. how are you? I am fine!  "
nodes = node_parser.get_nodes_from_documents([Document(text=text)])
print([x.text for x in nodes])

['hello. ', 'how are you? ', 'I am fine!  ']


In [6]:
print(nodes[1].metadata['window'])

hello.  how are you?  I am fine!  


In [7]:
text = "hello. foo bar. cat dog. mouse"

nodes = node_parser.get_nodes_from_documents([Document(text=text)])

print([x.text for x in nodes])

['hello. ', 'foo bar. ', 'cat dog. ', 'mouse']


In [8]:
print(nodes[0].metadata['window'])

hello.  foo bar.  cat dog. 


In [9]:
print(nodes[1].metadata['window'])

hello.  foo bar.  cat dog.  mouse


In [10]:
from llama_index.llms import OpenAI

llm = OpenAI(model='gpt-3.5-turbo', temperature=0.1)

In [11]:
from llama_index import ServiceContext

sentence_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    node_parser=node_parser
)

In [12]:
from llama_index import VectorStoreIndex

sentence_index = VectorStoreIndex.from_documents(
    [document], service_context=sentence_context
)

In [13]:
sentence_index.storage_context.persist(persist_dir="./sentence_index")

In [14]:
from llama_index import VectorStoreIndex, StorageContext, load_index_from_storage

if not os.path.exists("./sentence_index"):
    sentence_index = VectorStoreIndex.from_documents(
        [document], service_context=sentence_context
    )
    sentence_index.storage_context.persist(persist_dir="./sentence_index")
else:
    sentence_index = load_index_from_storage(
        StorageContext.from_defaults(persist_dir="./sentence_index"),
        service_context=sentence_context
    )

In [15]:
from llama_index.indices.postprocessor import MetadataReplacementPostProcessor

postproc = MetadataReplacementPostProcessor(
    target_metadata_key="window"
)

from llama_index.schema import NodeWithScore
from copy import deepcopy

scored_nodes = [NodeWithScore(node=x, score=1.0) for x in nodes]
nodes_old = [deepcopy(n) for n in nodes]

In [16]:
nodes_old[1].text

'foo bar. '

In [17]:
replpaced_nodes = postproc.postprocess_nodes(scored_nodes)
replpaced_nodes[1].text

'hello.  foo bar.  cat dog.  mouse'

In [18]:
from llama_index.indices.postprocessor import SentenceTransformerRerank

rerank = SentenceTransformerRerank(
    top_n = 2, model = "BAAI/bge-reranker-base"
)

from llama_index import QueryBundle
from llama_index.schema import TextNode, NodeWithScore

query = QueryBundle("I want a dog.")

scored_nodes = [
    NodeWithScore(node=TextNode(text="This is a cat."), score=0.6),
    NodeWithScore(node=TextNode(text="This is a dog"), score=0.4)
]

reranked_nodes = rerank.postprocess_nodes(
    scored_nodes,
    query_bundle=query
)

print([(x.text, x.score) for x in reranked_nodes])

[('This is a dog', 0.9182743), ('This is a cat.', 0.0009602243)]


In [19]:
sentenece_window_engine = sentence_index.as_query_engine(
    similarity_top_k=6, node_postprocessors=[postproc, rerank]
)

window_response = sentenece_window_engine.query(
    "What are the keys to building a career in AI?"
)

from llama_index.response.notebook_utils import display_response

display_response(window_response)

**`Final Response:`** The keys to building a career in AI are learning foundational technical skills, working on projects, and finding a job, all of which is supported by being part of a community.

In [20]:
import os
from llama_index import ServiceContext, VectorStoreIndex, StorageContext
from llama_index.node_parser import SentenceWindowNodeParser
from llama_index.indices.postprocessor import MetadataReplacementPostProcessor
from llama_index.indices.postprocessor import SentenceTransformerRerank
from llama_index import load_index_from_storage

def build_sentence_window_index(documents, llm, embed_model='local:BAAI/bge-small-en-v1.5', sentenece_window_size=3, save_dir="sentence_index"):
    node_parser = SentenceWindowNodeParser.from_defaults(
        window_size=sentenece_window_size,
        window_metadata_key="window",
        original_text_metadata_key="original_text"
    )

    sentence_context = ServiceContext.from_defaults(
        llm=llm,
        embed_model=embed_model,
        node_parser=node_parser
    )
    if not os.path.exists(save_dir):
        sentence_index = VectorStoreIndex.from_documents(
            documents, service_context=sentence_context
        )
        sentence_index.storage_context.persist(persist_dir=save_dir)
    else:
        sentence_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=save_dir),
            service_context=sentence_context,
        )

    return sentence_index

def get_sentence_window_query_engine(sentence_index, similarity_top_k=6, rerank_top_n=2):
    postproc=MetadataReplacementPostProcessor(target_metadata_key="window")
    rerank=SentenceTransformerRerank(top_n=rerank_top_n, model="BAAI/bge-reranker-base")
    sentenece_window_engine=sentence_index.as_query_engine(similarity_top_k=similarity_top_k, node_postprocessors=[postproc, rerank])
    return sentenece_window_engine

In [21]:
from llama_index.llms import OpenAI

index = build_sentence_window_index(
    [document],
    llm=OpenAI(model='gpt-3.5-turbo', temperature=0.1),
    save_dir="./sentence_index"
)

query_engine = get_sentence_window_query_engine(index, similarity_top_k=6)

In [22]:
eval_questions = []
with open('generated_questions.text', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        eval_questions.append(item)

In [23]:
from trulens_eval import Tru

def run_evals(eval_questions, tru_recorder, query_engine):
    for question in eval_questions:
        with tru_recorder as recording:
            response = query_engine.query(question)

In [24]:
from utils import get_prebuilt_trulens_recorder
from trulens_eval import Tru

Tru().reset_database()

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


In [25]:
sentence_index_1 = build_sentence_window_index(
    documents,
    llm=OpenAI(model='gpt-3.5-turbo', temperature=0.1),
    embed_model="local:BAAI/bge-small-en-v1.5",
    sentenece_window_size=1,
    save_dir="sentence_index_1"
)

sentence_window_engine_1 = get_sentence_window_query_engine(sentence_index_1)

tru_recorder_1 = get_prebuilt_trulens_recorder(sentence_window_engine_1, app_id="sentence window engine 1")

run_evals(eval_questions, tru_recorder_1, sentence_window_engine_1)

In [26]:
Tru().run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

RuntimeError: Dashboard failed to start in time. Please inspect dashboard logs for additional information.

In [28]:
eval_questions = []
with open('generated_questions.text', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        eval_questions.append(item)

sentence_index_3 = build_sentence_window_index(
    documents,
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),
    embed_model="local:BAAI/bge-small-en-v1.5",
    sentenece_window_size=3,
    save_dir="sentence_index_3",
)
sentence_window_engine_3 = get_sentence_window_query_engine(
    sentence_index_3
)

tru_recorder_3 = get_prebuilt_trulens_recorder(
    sentence_window_engine_3,
    app_id='sentence window engine 3'
)

run_evals(eval_questions, tru_recorder_3, sentence_window_engine_3)

In [29]:
Tru().run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

RuntimeError: Dashboard failed to start in time. Please inspect dashboard logs for additional information.